In [1]:
import requests
import pandas as pd
import datetime
import warnings
import sys
sys.path.append('C:/Users/User/Desktop/StockInfoHub')
from Shared_Modules.shared_functions import *
from Shared_Modules.shared_variables import *
from stock_data_collector_package.stock_data_collector_utils import *
url = "https://isin.twse.com.tw/isin/class_main.jsp?owncode=&stockname=&isincode=&market=1&issuetype=1&industry_code=&Page=1&chklike=Y"
response = requests.get(url)
listed = pd.read_html(response.text)[0]
listed.columns = listed.iloc[0,:]
listed = listed[["有價證券代號","有價證券名稱","市場別","產業別","公開發行/上市(櫃)/發行日"]]
listed = listed.iloc[1:]
# ============上櫃股票df============
urlTWO = "https://isin.twse.com.tw/isin/class_main.jsp?owncode=&stockname=&isincode=&market=2&issuetype=&industry_code=&Page=1&chklike=Y"
response = requests.get(urlTWO)
listedTWO = pd.read_html(response.text)[0]
listedTWO.columns = listedTWO.iloc[0,:]
listedTWO = listedTWO.loc[listedTWO['有價證券別'] == '股票']
listedTWO = listedTWO[["有價證券代號","有價證券名稱","市場別","產業別","公開發行/上市(櫃)/發行日"]]
# ============上市股票代號+.TW============
stock_1 = listed["有價證券代號"]
stock_num = stock_1.apply(lambda x: str(x) + ".TW")
taiwan_0050 = pd.Series(['0050.TW'], index=[0])
taiwan_weighted = pd.Series(['^TWII'], index=[0])
# print(stock_num)
# ============上櫃股票代號+.TWO============
stock_2 = listedTWO["有價證券代號"]
stock_num2 = stock_2.apply(lambda x: str(x) + ".TWO")
# print(stock_num2)
# ============concat全部股票代號============
stock_num = pd.concat([taiwan_weighted, taiwan_0050, stock_num, stock_num2], ignore_index=True)

Shared module initialized.


In [5]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# 设置目标网页的URL
for id in stock_num[2:]:
# for id in ['1101']:
    url = f'https://norway.twsthr.info/StockHolders.aspx?stock={id.split(".")[0]}'
    print(url)
    try:
        # 添加 headers
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}

        # 发送 HTTP 请求并获取网页内容
        response = requests.get(url, headers=headers)

        # 检查请求是否成功
        all_data = []
        if response.status_code == 200:
            # 使用 Beautiful Soup 解析网页内容
            soup = BeautifulSoup(response.text, 'html.parser')

            # 找到目标表格（这里使用id="Details"作为示例）
            table = soup.find('table', {'id': 'Details'})
            # 遍历表格的行和列，并提取数据
            for row in table.find_all('tr'):
                columns = row.find_all(['td'])
                row_data = [col.text.strip() for col in columns]
                all_data.append(row_data)
                # print('\t'.join(row_data))
        

        else:
            print(f'Failed to retrieve the webpage. Status code: {response.status_code}')
        # print(all_data)
        data = [i for i in all_data[1] if i != ''][:-1]
        data = [data[i*13:(i+1)*13] for i in range(int(len(data)/13))]
        columns_name = '資料日期	集保總張數	總股東人數	平均張數/人	>400張大股東持有張數	>400張大股東持有百分比	>400張大股東人數	400~600張人數	600~800張人數	800~1000張人數	>1000張人數	>1000張大股東持有百分比	收盤價'.split('\t')
        data_df = pd.DataFrame(data, columns=columns_name)
        data_df['stock_id'] = id
        data_df.to_excel(f'C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/股東人數資料/{id}.xlsx')
    except:
        print(id)
        continue

https://norway.twsthr.info/StockHolders.aspx?stock=1101
https://norway.twsthr.info/StockHolders.aspx?stock=1102
https://norway.twsthr.info/StockHolders.aspx?stock=1103
https://norway.twsthr.info/StockHolders.aspx?stock=1104
https://norway.twsthr.info/StockHolders.aspx?stock=1108
https://norway.twsthr.info/StockHolders.aspx?stock=1109
https://norway.twsthr.info/StockHolders.aspx?stock=1110
https://norway.twsthr.info/StockHolders.aspx?stock=1201
https://norway.twsthr.info/StockHolders.aspx?stock=1203
https://norway.twsthr.info/StockHolders.aspx?stock=1210
https://norway.twsthr.info/StockHolders.aspx?stock=1213
https://norway.twsthr.info/StockHolders.aspx?stock=1215
https://norway.twsthr.info/StockHolders.aspx?stock=1216
https://norway.twsthr.info/StockHolders.aspx?stock=1217
https://norway.twsthr.info/StockHolders.aspx?stock=1218
https://norway.twsthr.info/StockHolders.aspx?stock=1219
https://norway.twsthr.info/StockHolders.aspx?stock=1220
https://norway.twsthr.info/StockHolders.aspx?sto

In [17]:
data_df.head()

,資料日期,集保總張數,總股東人數,平均張數/人,>400張大股東持有張數,>400張大股東持有百分比,>400張大股東人數,400~600張人數,600~800張人數,800~1000張人數,>1000張人數,>1000張大股東持有百分比,收盤價
0,20231208,"25,932,071","1,227,754",21.12,"23,182,379",89.40,"2,531",483,282,215,1551,86.99,570.00
1,20231201,"25,932,071","1,220,827",21.24,"23,190,351",89.43,"2,526",479,285,210,1552,87.03,579.00
2,20231124,"25,932,071","1,218,912",21.27,"23,200,339",89.47,"2,530",478,285,213,1554,87.06,575.00
3,20231117,"25,932,071","1,225,855",21.15,"23,194,463",89.44,"2,527",475,280,214,1558,87.06,580.00
4,20231110,"25,932,071","1,259,864",20.58,"23,148,522",89.27,"2,528",486,275,214,1553,86.88,557.00


In [15]:
columns_name = '資料日期	集保總張數	總股東人數	平均張數/人	>400張大股東持有張數	>400張大股東持有百分比	>400張大股東人數	400~600張人數	600~800張人數	800~1000張人數	>1000張人數	>1000張大股東持有百分比	收盤價'
columns_name = columns_name.split('\t')
print(columns_name)

['資料日期', '集保總張數', '總股東人數', '平均張數/人', '>400張大股東持有張數', '>400張大股東持有百分比', '>400張大股東人數', '400~600張人數', '600~800張人數', '800~1000張人數', '>1000張人數', '>1000張大股東持有百分比', '收盤價']
